In [1]:
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU: NVIDIA GeForce RTX 2070 SUPER is available.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

# TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
# TinyLlama/TinyLlama-1.1B-Chat-v1.0
# 載入 HuggingFace 上的模型
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

In [8]:
# prompt 樣板設定測試
messages = [
    {   "role": "user", 
        "content": "Do you know what is the Monoploy?"
    }
]

outputs = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
outputs = outputs[0]["generated_text"]
# 設定 prompt 樣板，方便觀察輸出
chat_history = pipe.tokenizer.apply_chat_template(outputs, tokenize=False, add_generation_prompt=False)
print(chat_history)

<|user|>
Do you know what is the Monoploy?</s>
<|assistant|>
Yes, the Monoploy is a term used in marketing to refer to a single product or service that offers a unique selling proposition (USP) that is different from its competitors. The Monoploy is often created as a way to differentiate a brand from the competition, which helps to create a loyal customer base and increase market share. The term has been applied to various products and services, including products such as Apple's iPad, Nike's Air Jordan sneakers, and Amazon's Kindle e-readers.</s>



RAG

![image.png](MarkDownImage\RAG.png)

In [4]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores  import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

model_kwargs = {'device': 'cuda'}
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
    )
    

pdf_folder = "documents/monopoly"

pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

documents = []

for pdf_file in pdf_files:
    document = os.path.join(pdf_folder, pdf_file)
    pdf_loader = PyPDFLoader(document)
    documents.extend(pdf_loader.load())


In [5]:
vectordb= Chroma.from_documents(
    documents, 
    embedding,
    persist_directory="./knowledge-base"
)

In [6]:
def retrieve_documents(query, num_results=1):
    query_embedding = embedding.embed_query(query)
    results = vectordb.similarity_search_by_vector(query_embedding, k=num_results)
    retrieved_docs = [result.page_content for result in results]
    return retrieved_docs

def generate_answer(query, retrieved_docs):
    
    retrieved_docs_text = "\n".join(retrieved_docs)
    
    content = f"""
Given the following documents, answer the user's question as accurately as possible:

Documents:
{retrieved_docs_text}

Question:
{query}
"""
    messages = [
        {   "role": "system", 
            "content": content,
        }
    ]
    result = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    outputs = result[0]['generated_text']
    
    assistant_reply = next((msg["content"] for msg in outputs if msg["role"] == "assistant"), "")
    
    return assistant_reply, retrieved_docs_text


def process_query(query, num_results=1):

    retrieved_docs = retrieve_documents(query, num_results=num_results)
    answer, retrieved_docs_text = generate_answer(query, retrieved_docs)
    
    return answer, retrieved_docs_text


In [ ]:
query = "Where the Monopoly start from and what is the rules?"
answer, retrieved_docs_text = process_query(query)

In [11]:
print(answer)

The Monopoly start from the Banker's corner marked "Go" and move in the direction of the arrow. The Banker's token is held on the corner marked "GO" when the turn to play passes to the left. Two or more tokens may rest on the same space at the same time, according to the space they reach. The Banker may pay $200 salary to each player who lands on "GO" by throw of the dice or draw a card. Players can buy, rent or pay taxes, draw a Chance or Community Chest card, "Go to Jail", etc.


In [12]:
print(retrieved_docs_text)

 
Monopoly Rules 
 
HISTORY… PARKER BROTHERS Real Estate Trading Game MONOPOLY© was invented during the Great Depression by Charles 
B. Darrow of Germantown Pennsylvania. Mr. Darrow, like. many other Americans, was unemployed at the time and he worked out the 
details of the game primarily to amuse himself during this period. Prior to the Depression, Darrow and his wife vacationed in the resort 
town of Atlantic City, New Jersey. When it came to naming the streets on the game board, Darrow naturally adopted those of his favorite 
vacation spot. 
The first games were handmade. Darrow gave them to friends and sold a few through a Philadelphia department store. As the demand for 
the game grew, Darrow could not keep up with the orders and arranged for Parker Brothers to take over the game. 
Since 1935 when Parker Brothers acquired the rights to the game, it has become the leading proprietary game not only in the United States 
but throughout the Western world. the game is published under 